In [2]:
parent_folder = '/Users/username/Desktop/ML/Recommendations/arcane/'
from hydra import compose, initialize
# with initialize(config_path="../../conf"):
#     # Compose the configuration
#     cfg = compose(config_name="ClustersResetService.yaml")

import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/username/Desktop/ML/Recommendations/arcane/')

from src._utils import load_bertopic_model_from_hf

/Users/username/anaconda3/envs/recoenv/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/username/anaconda3/envs/recoenv/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/username/anaconda3/envs/recoenv/lib/python3.11/site-pac

In [3]:
bertopic = load_bertopic_model_from_hf()

# len(bertopic.topic_embeddings_)

# story_embeddings = {i: list([float(x) for x in bertopic.topic_embeddings_[i+1]]) for i in range(-1,len(bertopic.topic_embeddings_)-1)}

In [30]:
bertopic.save?

Signature:
bertopic.save(
    path,
    serialization: Literal['safetensors', 'pickle', 'pytorch'] = 'pickle',
    save_embedding_model: Union[bool, str] = True,
    save_ctfidf: bool = False,
)
Docstring:
Saves the model to the specified path or folder

When saving the model, make sure to also keep track of the versions
of dependencies and Python used. Loading and saving the model should
be done using the same dependencies and Python. Moreover, models
saved in one version of BERTopic should not be loaded in other versions.

Arguments:
    path: If `serialization` is 'safetensors' or `pytorch`, this is a directory.
          If `serialization` is `pickle`, then this is file.
    serialization: If `pickle`, the entire model will be pickled. If `safetensors`
                   or `pytorch` the model will be saved without the embedding,
                   dimensionality reduction, and clustering algorithms.
                   This is a very efficient format and typically advised.
    save

In [4]:
from sql.clustering.ClusteringSQL import ClusteringSQL

In [5]:
latest_run = ClusteringSQL.get_latest_run_id()

In [6]:
# with PostgresDatabaseOperation() as cursor:
#     sql = """
#             INSERT INTO story_embeddings (story_id, story_embedding, clustering_run_id)
#             VALUES (%s, %s, %s)
#             ON CONFLICT (story_id, clustering_run_id)
#             DO UPDATE SET 
#             story_embedding = EXCLUDED.story_embedding;
#     """
#     for story_id in story_embeddings.keys():
#         cursor.execute(sql, (story_id, story_embeddings[story_id], latest_run))

In [6]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [7]:
with PostgresDatabaseOperation() as cursor:
    sql = 'SELECT article_id, embedding FROM embeddings ORDER BY created_at DESC LIMIT 10'
    cursor.execute(sql)
    result = cursor.fetchall()

In [8]:
emb = {x[0]: [float(k) for k in x[1]] for x in result}

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
emb.keys()

dict_keys(['6538f5531e5cc42b1b1444b4', '6538f54f1e5cc42b1b1444a8', '6538f5511e5cc42b1b1444ab', '6538f5531e5cc42b1b1444b8', '6538f5511e5cc42b1b1444ac', '6538f5501e5cc42b1b1444aa', '6538f5511e5cc42b1b1444b0', '6538f5511e5cc42b1b1444af', '6538f5521e5cc42b1b1444b1', '6538f54b1e5cc42b1b1444a1'])

In [12]:
from sql.clustering.ClusteringSQL import ClusteringSQL

In [13]:
story_embeddings = ClusteringSQL.get_all_story_embeddings()

In [14]:
sim_matrix = cosine_similarity([emb['6538f5511e5cc42b1b1444af']], list(story_embeddings.values()))

In [16]:
np.argsort(sim_matrix)[0][::-1]

array([ 72,   9,   0,  20,  11,  30,   4,  45,  36,  53,  22,   6,  46,
        75,  34,  12,  48, 104,   2,  57,  68,   5,  47,  71,  43,  52,
        39,  13,   1,  55,  67,  33,  65,  78,  83,   3, 100,  96, 108,
        19,  26,  29,  77,   8,  49,  37,  99,  61,  73,  95,  58,  27,
        40,  18,  79,  25,  35,  50, 112,  28,  41,  91,  51, 109,  59,
        86, 101, 102,  17,  74,  60, 107,  90,  88,  24,  93,  31,  14,
        97,  32,   7,  54,  89,  64, 111,  38, 105,  63,  44,  81,  84,
        21,  56,  76,  94,  80, 103, 110,  69, 106,  70,  82,  87,  92,
        62,  10,  66,  98,  42,  16,  85,  23,  15])

In [11]:
import numpy as np

In [19]:
from src.articles.ArticleService import ArticleService

In [20]:
art = ArticleService.get_Article('6538f5511e5cc42b1b1444af')

In [21]:
art.full_content

'PowerGrid to announce dividend Q2 results on November 7 fixes record date  Mint:  Power Grid Corporation of India the state-run power transmission company will announce its financial results for the second quarter of FY24 on November 7 Tuesday. The companys board will also declare an interim dividend in the quarterly earnings. PowerGrids board of directors will meet on November 7 to announce the earnings for the quarter ended September 2023 and an interim dividend it said in a release. The company has also fixed 16th November 2023 as the record date for the dividend it added. (Exciting news Mint is now on WhatsApp Channels Subscribe today by clicking the link and stay updated with the latest financial insights Click here ) ... a Meeting of Board of Directors of POWERGRID is scheduled to be held on Tuesday 7th November 2023 to consider and approve amongst other items of Agenda the Unaudited Financial Results of the Company for the quarter and half year ended 30th September 2023 after t

In [24]:
story, probabilities = bertopic.transform(art.full_content)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
len(probabilities)

1

In [32]:
bertopic.probabilities_

In [29]:
probabilities/np.sum(probabilities)

array([[0.01013445, 0.00913752, 0.00929994, 0.00906823, 0.00955794,
        0.00927624, 0.00943177, 0.00861935, 0.00900229, 0.01016088,
        0.00806689, 0.00970106, 0.00935434, 0.00917096, 0.00864661,
        0.00724174, 0.00787508, 0.00872756, 0.00884577, 0.0090534 ,
        0.01009753, 0.00843978, 0.00944846, 0.00735803, 0.00869124,
        0.00883011, 0.00904457, 0.00885108, 0.00879388, 0.00901111,
        0.00959751, 0.00866394, 0.00863887, 0.00909585, 0.00935624,
        0.00881877, 0.00946455, 0.00899415, 0.00854697, 0.00920905,
        0.00884671, 0.00879101, 0.00791075, 0.00924157, 0.00848669,
        0.00947602, 0.00941952, 0.00927273, 0.00934367, 0.00899566,
        0.00881873, 0.00878701, 0.00922298, 0.00944989, 0.00861321,
        0.0091172 , 0.00842922, 0.00929889, 0.00892131, 0.00875411,
        0.00872094, 0.00896726, 0.00807443, 0.00849578, 0.00856595,
        0.00909321, 0.00800432, 0.00909839, 0.00929489, 0.00832921,
        0.00827075, 0.00924606, 0.01083172, 0.00

In [25]:
probabilities

array([[0.7708198 , 0.69499433, 0.707348  , 0.6897237 , 0.72697085,
        0.70554525, 0.7173747 , 0.65558225, 0.6847087 , 0.7728302 ,
        0.613563  , 0.7378565 , 0.7114855 , 0.6975376 , 0.6576555 ,
        0.5508023 , 0.5989735 , 0.663813  , 0.672804  , 0.6885962 ,
        0.76801217, 0.6419248 , 0.71864367, 0.5596472 , 0.66104996,
        0.6716125 , 0.6879246 , 0.67320734, 0.66885716, 0.6853792 ,
        0.7299807 , 0.6589739 , 0.65706694, 0.6918249 , 0.71162975,
        0.6707501 , 0.71986777, 0.6840894 , 0.6500775 , 0.70043427,
        0.67287517, 0.6686387 , 0.6016866 , 0.70290804, 0.6454922 ,
        0.72073996, 0.71644306, 0.7052778 , 0.71067375, 0.684204  ,
        0.6707468 , 0.6683346 , 0.7014938 , 0.718753  , 0.65511525,
        0.69344884, 0.6411214 , 0.7072676 , 0.6785494 , 0.6658325 ,
        0.6633096 , 0.68204427, 0.61413646, 0.6461841 , 0.6515208 ,
        0.69162405, 0.6088034 , 0.6920183 , 0.7069638 , 0.63351476,
        0.6290681 , 0.7032495 , 0.8238541 , 0.68

In [ ]:
sim_matrix

In [23]:
sim_matrix

array([[0.77082022, 0.69499476, 0.70734843, 0.68972419, 0.7269712 ,
        0.70554563, 0.71737527, 0.65558253, 0.68470908, 0.77283045,
        0.61356334, 0.73785692, 0.711486  , 0.69753794, 0.65765581,
        0.55080274, 0.59897392, 0.66381355, 0.67280446, 0.68859658,
        0.76801257, 0.64192531, 0.71864413, 0.55964749, 0.66105032,
        0.67161279, 0.68792496, 0.67320774, 0.66885745, 0.68537964,
        0.72998106, 0.65897434, 0.65706727, 0.69182519, 0.71163003,
        0.67075057, 0.71986833, 0.68408984, 0.65007806, 0.70043454,
        0.67287558, 0.66863916, 0.60168704, 0.70290838, 0.64549252,
        0.72074039, 0.71644342, 0.70527816, 0.71067424, 0.68420434,
        0.67074721, 0.66833503, 0.70149425, 0.71875331, 0.65511562,
        0.69344925, 0.64112177, 0.70726803, 0.67854977, 0.66583288,
        0.66330997, 0.68204458, 0.61413682, 0.64618449, 0.65152127,
        0.69162447, 0.60880379, 0.69201856, 0.70696416, 0.63351519,
        0.62906844, 0.70325003, 0.82385453, 0.68

In [28]:
np.allclose(probabilities, sim_matrix)

True